#Chapter 14. Local effects and mutable state

### _more mature concept of 'referential transparency'_

## 14.1. Purely functional mutable state

- 순수 FP에서는 mutable state를 허용하지 않는다고 생각해 왔지만, 사실은 그렇지 않다!
- "참조 투명성"의 정의부터 살펴보자.
#### "참조 투명성(referential transparency)"의 정의
  * **어떠한 표현식 _e_가 프로그램 _p_안에서 쓰일 때,**
  * **모든 _e_가 나오는 부분을 _e_의 평가결과 값으로 교체해도 **
  * **_p_의 전체 의미가 달라지지 않을 때 _e_를 참조(에 대해) 투명하다고 한다.**
    * **모든 참조 투명한 x값에 대해 어떤 함수 f(x)가 참조 투명하면 f를 순수(pure)하다고 한다.**

 * **_pure_**한 quicksort 함수 정의

In [21]:
def quicksort(xs: List[Int]): List[Int] = if (xs.isEmpty) xs else {
    /* list => mutable array (swap, partitioning) => list */
    val arr = xs.toArray
    def swap(x: Int, y: Int) = {
      val tmp = arr(x)
      arr(x) = arr(y)
      arr(y) = tmp
    }
    def partition(l: Int, r: Int, pivot: Int) = {
      val pivotVal = arr(pivot)
      swap(pivot, r)
      var j = l
      for (i <- l until r) if (arr(i) < pivotVal) {
        swap(i, j)
        j += 1
      }
      swap(j, r)
      j
    }
    def qs(l: Int, r: Int): Unit = if (l < r) {
      val pi = partition(l, r, l + (r - l) / 2)
      qs(l, pi - 1)
      qs(pi + 1, r)
    }
    qs(0, arr.length - 1)
    arr.toList
  }

In [22]:
val mylist = List(3,1,2,5,7,9,0,4,8,6)

List(3, 1, 2, 5, 7, 9, 0, 4, 8, 6)

In [23]:
quicksort(mylist)

List(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

In [24]:
mylist

List(3, 1, 2, 5, 7, 9, 0, 4, 8, 6)

<img src="files/imgs/01.png" align="left" style="border:1px solid blue;">

##14.2. A data type to enforce scoping of side effects

- quicksort() 함수에서 입력 리스트를 직접 바꿀 수도 있었을 것이다. 그랬다면 외부에서 side effect 관찰 가능.
- 스칼라의 타입시스템을 활용해서 side-effect의 스코프를 (local로) 강제하는 데이터 타입을 만들어보자.
  * **_성능과 표기상의 편의를 생각하면 이런 짓을 하는 비용이 크다. 쓸지 말지는 알아서 결정할 것.._**

###14.2.1. A little language for scoped mutation

- 새로운 데이터 타입을 이용해서 mutation을 발생시키는 경우 보장되어야 할 점.
  - mutable object에 대한 참조를 들고 있는 경우, 외부에서는 우리가 mutation시키는 것을 알 수 없어야 함.
  - mutable object는 그것이 생성된 스코프를 벗어나는 경우 절대 참조될 수 없어야 함.

- State[S,A] 모나드
  - mutable state를 다루기 편한 타입임.
  - S => (A,S) 형식의 함수였음을 기억할 것.

- ST(State Token) 모나드
  - 'S'라는 것은 상태 변경(mutation)을 할 수 있다는 토큰임.

In [25]:
sealed trait ST[S,A] { self =>
  protected def run(s: S): (A,S)
  def map[B](f: A => B): ST[S,B] = new ST[S,B] {
    def run(s: S) = {
      val (a, s1) = self.run(s)
      (f(a), s1)
    }
  }
  def flatMap[B](f: A => ST[S,B]): ST[S,B] = new ST[S,B] {
    def run(s: S) = {
      val (a, s1) = self.run(s)
      f(a).run(s1)
    }
  }
}

// below is for 14.2.3. elsewhere generates err...
object ST {
  def apply[S,A](a: => A) = {
    lazy val memo = a
    new ST[S,A] {
      def run(s: S) = (memo, s)
    }
  }
  def runST[A](st: RunnableST[A]): A =
    st[Null].run(null)._1
}

trait RunnableST[A] {
  def apply[S]: ST[S,A]
}

###14.2.2. An algebra of mutable references

- mutable memory cells
  - new mutable cell 할당
  - mutable cell에 쓰기
  - mutable cell 읽기

- protected var에 대한 wrapper

In [26]:
sealed trait STRef[S,A] {
  protected var cell: A
  def read: ST[S,A] = ST(cell)
  def write(a: => A): ST[S,Unit] = new ST[S,Unit] {
    def run(s: S) = {
      cell = a
      ((), s)
    }
  }
}

object STRef {
  def apply[S,A](a: A): ST[S, STRef[S,A]] = ST(new STRef[S,A] {
    var cell = a
  })
}

- 'S'는 cell 변수의 타입도 아니고, 실제로 쓰이지도 않지만 상태 변경을 위해서는 반드시 'S' 타입의 값을 제공해야 한다.
- 이런 의미에서 'S'를 mutation을 허가할 수 있는 일종의 '토큰'으로 생각할 수 있음.

````
for {   
    r1 <- STRef[Nothing,Int](1)   
    r2 <- STRef[Nothing,Int](1)   
    x <- r1.read   
    y <- r2.read   
    _ <- r1.write(y+1)   
    _ <- r2.write(x+1)   
    a <- r1.read
    b <- r2.read
} yield(a,b)

// 아직 동작하지 않음..(run()이 여전히 protected이고, Nothing으로는 '값'을 전달할 수 없으므로..라고 책에선 얘기함..)
````

###14.2.3. Running mutable state actions

- ST로부터 STRef를 바로 접근할 수 없음을 보장해야 한다.
````
  * ST[S, STRef[S, Int]] (not safe to run)
  * ST[S, Int] (completely safe to run)  => var쪽에 type S가 개입되지 않기 때문...
````
- **ST[S, T]에서 T가 S타입을 관여시키는 타입이면 ST를 실행할 수 없게 하자..**
  - 왜냐하면, ST에서 STRef를 바로 꺼낼 수 있으면 내부 mutable object에 접근하게 할 수도 있으므로..

In [36]:
val p = new RunnableST[(Int, Int)] {   
    def apply[S] = for {     
        r1 <- STRef(1)     
        r2 <- STRef(2)     
        x <- r1.read     
        y <- r2.read     
        _ <- r1.write(y+1)     
        _ <- r2.write(x+1)     
        a <- r1.read     
        b <- r2.read   
    } yield (a,b) 
}

cmd32$$anon$1@ecd86ae

In [37]:
val r = ST.runST(p)

(3,2)

- 아래처럼 바로 STRef를 접근해서 쓰려는 경우는 type system에 의해서 에러가 난다.

In [38]:
new RunnableST[STRef[Nothing, Int]] {
    def apply[S] = STRef(1)
}

: 

###14.2.4. Mutable arrays

- mutable var 하나로는 별로 유용하지 않음.

In [30]:
// Scala requires an implicit Manifest for constructing arrays.
sealed abstract class STArray[S,A](implicit manifest: Manifest[A]) {
  protected def value: Array[A]
  def size: ST[S,Int] = ST(value.size)
  // Write a value at the give index of the array
  def write(i: Int, a: A): ST[S,Unit] = new ST[S,Unit] {
    def run(s: S) = {
      value(i) = a
      ((), s)
    }
  }
  // Read the value at the given index of the array
  def read(i: Int): ST[S,A] = ST(value(i))
  // Turn the array into an immutable list
  def freeze: ST[S,List[A]] = ST(value.toList)

  def fill(xs: Map[Int,A]): ST[S,Unit] =
    xs.foldRight(ST[S,Unit](())) {
    case ((k, v), st) => st flatMap (_ => write(k, v))
  }

/////////////////////////////////////////////////////////////////////
  // below for 14.2.5. 
  def swap(i: Int, j: Int): ST[S,Unit] = for {
    x <- read(i)
    y <- read(j)
    _ <- write(i, y)
    _ <- write(j, x)
  } yield ()
}

object STArray {
  // Construct an array of the given size filled with the value v
  def apply[S,A:Manifest](sz: Int, v: A): ST[S, STArray[S,A]] =
    ST(new STArray[S,A] {
      lazy val value = Array.fill(sz)(v)
    })

  def fromList[S,A:Manifest](xs: List[A]): ST[S, STArray[S,A]] =
    ST(new STArray[S,A] {
      lazy val value = xs.toArray
    })
}


- STRef와 마찬가지로 STArray를 ST action에 패키징해서 처리하게 하려함..
- Array 원소를 읽는 것 자체도 ST에 S태그가 들어간 것으로 처리함.

### 14.2.5. A purely functional in-place quicksort

In [32]:

object Immutable {
  def noop[S] = ST[S,Unit](())

    def partition[S](a: STArray[S,Int], l: Int, r: Int, pivot: Int): ST[S,Int] = for {
      vp <- a.read(pivot)
      _ <- a.swap(pivot, r)
      j <- STRef(l)
      _ <- (l until r).foldLeft(noop[S])((s, i) => for {
        _ <- s
        vi <- a.read(i)
        _  <- if (vi < vp) (for {
          vj <- j.read
          _  <- a.swap(i, vj)
          _  <- j.write(vj + 1)
        } yield ()) else noop[S]
      } yield ())
      x <- j.read
      _ <- a.swap(x, r)
    } yield x

    def qs[S](a: STArray[S,Int], l: Int, r: Int): ST[S, Unit] = if (l < r) for {
      pi <- partition(a, l, r, l + (r - l) / 2)
      _ <- qs(a, l, pi - 1)
      _ <- qs(a, pi + 1, r)
    } yield () else noop[S]


  def quicksort(xs: List[Int]): List[Int] =
    if (xs.isEmpty) xs else ST.runST(new RunnableST[List[Int]] {
      def apply[S] = for {
        arr    <- STArray.fromList(xs)
        size   <- arr.size
        _      <- qs(arr, 0, size - 1)
        sorted <- arr.freeze
      } yield sorted
  })
}

In [33]:
mylist

List(3, 1, 2, 5, 7, 9, 0, 4, 8, 6)

In [34]:
Immutable.quicksort(mylist)

List(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

In [35]:
mylist

List(3, 1, 2, 5, 7, 9, 0, 4, 8, 6)

## 14.3. Purity is contextual

In [39]:
case class Foo(s: String)

In [40]:
val b = Foo("hello") == Foo("hello")

true

In [41]:
val c = Foo("hello") eq Foo("hello")

false

- 참조 투명성 관점에서 보자면, Scala의 모든 데이터 생성자는 (eq 연산자에 대해서) side-effect가 있는 셈.(메모리 할당하고, 기타 등등..)
- 하지만, eq 연산자만 그런 것이고, 대부분의 프로그램 코드에서는 문제가 되지 않음.
- 즉, side-effect라는 것도 크게 보면 해당 expression이 어떤 맥락에서 사용되고 있는지를 따져봐서, 특정 맥락에서 "관측이 되지 않으면"참조 투명성이 있다고 봐도 무방하겠다..

###14.3.1. What counts as a side-effect

In [42]:
def timesTwo(x: Int) = {
    if (x < 0) println("Got a negative number")
    x*2
}

In [43]:
timesTwo(1)

2

- 프로그램상에서 timesTwo(1)이 출현하는 곳을 모두 '2'로 바꾸면...?
  - 이 경우 프로그램의 의미가 달라졌다고 할 것인가??
- **결국, 어떤 것이 유의미한(side-effect를 고려하면서 끝까지 추적가능하게 만들) effect인가를 (프로그래머 스스로) 결정해야 한다.**

# *"But this isn’t the end of the road."*
# *"You’re limited only by your imagination and the expressiveness of Scala’s types."*